# Lab Four: The Multi-Layer Perceptron
 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Load, Split, and Balance

#### 1.1 Load Data

In [29]:
import pandas as pd
import numpy as np

# Load the data into memory and save it to a pandas data frame.
df = pd.read_csv('census_dataset/acs2017_census_tract_data.csv')

# Remove any observations that having missing data.
df = df.dropna()

# Encode any string data as integers for now. 
# df['State'] = df['State'].astype(int)
# df['County'] = df['County'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72718 entries, 0 to 74000
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TractId           72718 non-null  int64  
 1   State             72718 non-null  object 
 2   County            72718 non-null  object 
 3   TotalPop          72718 non-null  int64  
 4   Men               72718 non-null  int64  
 5   Women             72718 non-null  int64  
 6   Hispanic          72718 non-null  float64
 7   White             72718 non-null  float64
 8   Black             72718 non-null  float64
 9   Native            72718 non-null  float64
 10  Asian             72718 non-null  float64
 11  Pacific           72718 non-null  float64
 12  VotingAgeCitizen  72718 non-null  int64  
 13  Income            72718 non-null  float64
 14  IncomeErr         72718 non-null  float64
 15  IncomePerCap      72718 non-null  float64
 16  IncomePerCapErr   72718 non-null  float6

#### 1.2 Split Dataset

In [32]:
# Divide your data into training and testing data using an 80% training 
# and 20% testing split. Use the cross validation modules that are part 
# of scikit-learn.
from sklearn.model_selection import train_test_split

X = df.drop(columns = ['ChildPoverty' ])
y = df['ChildPoverty']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

unique_ytrain, counts_ytrain = np.unique(y_train, return_counts=True)
print(np.asarray((unique_ytrain, counts_ytrain)).T)

[[0.000e+00 3.907e+03]
 [1.000e-01 2.100e+01]
 [2.000e-01 2.400e+01]
 ...
 [9.740e+01 1.000e+00]
 [9.830e+01 1.000e+00]
 [1.000e+02 4.800e+01]]


#### 1.3 Balance

In [60]:
print(len(y),len(y)/2)
threshold = 16.3
print(len(y[y<threshold]))
print(len(y[y>=threshold]))

# all factors are {1, 2, 103, 206, 353, 706, 36359, 72718}

72718 36359.0
36239
36479


Should balancing of the dataset be done for both the training and testing set? Explain.

### 2. Pre-proccesing

### 3. Modeling

### 4. Adaptive momentum (AdaM) 